# <font style="color:blue">Project 2: Kaggle Competition - Classification</font>

#### Maximum Points: 100

<div>
    <table>
        <tr><td><h3>Sr. no.</h3></td> <td><h3>Section</h3></td> <td><h3>Points</h3></td> </tr>
        <tr><td><h3>1</h3></td> <td><h3>Data Loader</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>2</h3></td> <td><h3>Configuration</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>3</h3></td> <td><h3>Evaluation Metric</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>4</h3></td> <td><h3>Train and Validation</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>5</h3></td> <td><h3>Model</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>6</h3></td> <td><h3>Utils</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>7</h3></td> <td><h3>Experiment</h3></td><td><h3>5</h3></td> </tr>
        <tr><td><h3>8</h3></td> <td><h3>TensorBoard Log</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>9</h3></td> <td><h3>Kaggle Profile Link</h3></td> <td><h3>50</h3></td> </tr>
    </table>
</div>


## <font style="color:green">1. Data Loader [10 Points]</font>

In this section, you have to write a class or methods, which will be used to get training and validation data loader.

You need to write a custom dataset class to load data.

**Note; There is   no separate validation data. , You will thus have to create your own validation set, by dividing the train data into train and validation data. Usually, we do 80:20 ratio for train and validation, respectively.**


For example:

```python
class KenyanFood13Dataset(Dataset):
    """
    
    """
    
    def __init__(self, *args):
    ....
    ...
    
    def __getitem__(self, idx):
    ...
    ...
    
    
```

```
def get_data(args1, *agrs):
    ....
    ....
    return train_loader, test_loader
```

In [1]:
import torch
import lightning as L
import numpy as np
import pandas as pd

from typing import TypeAlias, Generator

from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


RGB: TypeAlias = tuple[int, int, int]

IMAGE_SIZE: int = 224


class KenyanFood13Dataset(Dataset):
    """
    KenyanFood13Dataset is Dataset class for the Kenyan Food 13 dataset.
    """

    def __init__(
        self,
        data_root: str,
        train=True,
        transform=lambda x: x,
        random_state: int = 42,
    ) -> None:
        """
        Initializes the dataset.

        Args:
            train (bool): True if training data, False if validation data
        """
        self.data_csv_path = f"{data_root}/train.csv"
        self.images_dir = f"{data_root}/images/images"

        self.train = train
        self.transform = transform

        self.df = pd.read_csv(self.data_csv_path)
        data_np = self.df.to_numpy()
        X = data_np[:, 0]
        y = data_np[:, 1]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X,
            y,
            test_size=0.2,
            random_state=random_state,
        )

        if self.train:
            self.X, self.y = self.X_train, self.y_train
        else:
            self.X, self.y = self.X_test, self.y_test

        self.class_to_idx = {c: i for i, c in enumerate(np.unique(y))}

    def __len__(self) -> int:
        """
        Returns the length of the dataset.

        Returns:
            int: length of the dataset
        """
        return len(self.X)

    def __getitem__(self, idx) -> tuple[Image.Image, int] | Image.Image:
        """
        Returns the image and label for training dataset and only the image for validation dataset.

        Args:
            idx (int): index of the item
        Returns:
            tuple[Image.Image, int]: (image, label)
        """
        img_id, img_class = self.X[idx], self.y[idx]
        img = Image.open(f"{self.images_dir}/{img_id}.jpg")

        return self.transform(img), self.class_to_idx[img_class]


class KenyanFood13DataModule(L.LightningDataModule):
    """
    KenyanFood13DataModule is LightningDataModule class for the Kenyan Food 13 dataset.
    """

    def __init__(
        self,
        data_root: str,
        train_batch_size: int = 32,
        val_batch_size: int = 32,
        num_workers: int = 4,
        use_augmentation: bool = False,
    ) -> None:
        """
        Initializes KenyanFood13DataModule

        Args:
            data_root (str): path to the data root
            train_batch_size (int, optional): batch size for training. Defaults to 32.
            val_batch_size (int, optional): batch size for validation. Defaults to 32.
            num_workers (int, optional): number of workers. Defaults to 4.
            use_augmentation (bool, optional): whether to use data augmentation. Defaults to False.

        Returns:
            None
        """
        super().__init__()

        self.data_root = data_root
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.num_workers = num_workers
        self.use_augmentation = use_augmentation

        mean, std = get_mean_and_std(data_root)
        self.common_transforms = transforms.Compose(
            [
                transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ]
        )

        if use_augmentation:
            self.train_transforms = transforms.Compose(
                [
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomVerticalFlip(),
                    transforms.RandomRotation(degrees=(0, 30)),
                    transforms.RandomGrayscale(p=0.1),
                    transforms.RandomPerspective(
                        distortion_scale=0.2,
                        p=0.5,
                    ),
                    self.common_transforms,
                    transforms.RandomErasing(),
                ]
            )
        else:
            self.train_transforms = self.common_transforms

    def setup(self, stage: str | None = None) -> None:
        """
        Loads the train and validation datasets.

        Args:
            stage (str | None, optional): stage. Defaults to None.

        Returns:
            None
        """
        self.train_dataset = KenyanFood13Dataset(
            data_root=self.data_root,
            train=True,
            transform=self.train_transforms,
        )

        self.val_dataset = KenyanFood13Dataset(
            data_root=self.data_root,
            train=False,
            transform=self.common_transforms,
        )

    def train_dataloader(self) -> DataLoader:
        """
        Creates and returns the training dataloader.

        Returns:
            DataLoader: training dataloader
        """
        return DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
            num_workers=self.num_workers,
            shuffle=True,
        )

    def val_dataloader(self) -> DataLoader:
        """
        Creates and returns the validation dataloader.

        Returns:
            DataLoader: validation dataloader
        """
        return DataLoader(
            self.val_dataset,
            batch_size=self.val_batch_size,
            num_workers=self.num_workers,
            shuffle=False,
        )


def get_mean_and_std(data_root: str) -> tuple[RGB, RGB]:
    """
    Computes the mean and standard deviation of the train dataset.

    Returns:
        tuple[RGB, RGB]: (mean, std)
    """
    preprocess_transforms = transforms.Compose(
        [
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor(),
        ]
    )

    train_dataset = KenyanFood13Dataset(
        data_root=data_root,
        train=True,
        transform=preprocess_transforms,
    )

    dataloader = DataLoader(
        train_dataset,
        batch_size=32,
        num_workers=4,
        shuffle=False,
    )

    mean = torch.zeros(3)
    std = torch.zeros(3)
    for imgs, _ in dataloader:
        for i in range(3):
            mean[i] += imgs[:, i, :, :].mean()
            std[i] += imgs[:, i, :, :].std()

    mean.div_(len(dataloader))
    std.div_(len(dataloader))

    return mean, std


def kaggle_test_images(
    data_root: str,
) -> Generator[tuple[torch.Tensor, str], None, None]:
    """
    Gets the test images from the Kaggle dataset.

    Returns:
        tuple[torch.Tensor, torch.Tensor]: (image_names, images)
    """
    data_csv_path = f"{data_root}/test.csv"
    images_dir = f"{data_root}/images/images"

    df = pd.read_csv(data_csv_path)
    image_names = df.to_numpy()[:, 0]

    mean, std = get_mean_and_std(data_root)
    common_transforms = transforms.Compose(
        [
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ]
    )

    for image_name in image_names:
        image = Image.open(f"{images_dir}/{image_name}.jpg")
        yield common_transforms(image), image_name


/home/kamal/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## <font style="color:green">2. Configuration [5 Points]</font>

**Define your configuration here.**

For example:


```python
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 10 
    epochs_count: int = 50  
    init_learning_rate: float = 0.1  # initial learning rate for lr scheduler
    log_interval: int = 5  
    test_interval: int = 1  
    data_root: str = "/kaggle/input/opencv-pytorch-classification-project-2/" 
    num_workers: int = 2  
    device: str = 'cuda'  
    
```

In [2]:
from dataclasses import dataclass


@dataclass
class TrainingConfiguration:
    """
    Describes configuration of the training process
    """

    batch_size: int = 32
    epochs_count: int = 50
    init_learning_rate: float = 0.001  # initial learning rate for lr scheduler
    log_interval: int = 5
    test_interval: int = 1
    data_root: str = "./data"
    num_workers: int = 4
    device: str = "cuda"
    early_stopping_patience: int = 5


## <font style="color:green">3. Evaluation Metric [10 Points]</font>

**Define methods or classes that will be used in model evaluation. For example, accuracy, f1-score etc.**

In [3]:
import torch

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def get_metrics(
    y_true: torch.Tensor,
    y_pred: torch.Tensor,
) -> dict[str, float]:
    """
    Calculates and returns the metrics.

    Args:
        y_true (torch.Tensor): true labels
        y_pred (torch.Tensor): predicted labels
    Returns:
        dict[str, float]: metrics (accuracy, precision, recall, f1)
    """
    y_true_np = y_true.cpu().numpy()
    y_pred_np = y_pred.cpu().numpy()

    return {
        "accuracy": accuracy_score(y_true_np, y_pred_np),
        "precision": precision_score(y_true_np, y_pred_np, average="macro"),
        "recall": recall_score(y_true_np, y_pred_np, average="macro"),
        "f1": f1_score(y_true_np, y_pred_np, average="macro"),
    }


## <font style="color:green">4. Train and Validation [5 Points]</font>


**Write the methods or classes to be used for training and validation.**

In [4]:
import lightning as L

from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

from learn_lib.config import TrainingConfiguration


def train(
    model: L.LightningModule,
    data_module: L.LightningDataModule,
    training_configuration: TrainingConfiguration,
) -> None:
    early_stopping = EarlyStopping(
        monitor="val/acc",
        mode="max",
        verbose=True,
        patience=training_configuration.early_stopping_patience,
    )

    model_checkpoint = ModelCheckpoint(
        monitor="val/acc",
        mode="max",
        dirpath="checkpoints/",
        filename="model-{epoch:02d}",
        auto_insert_metric_name=False,
        verbose=True,
    )

    trainer = L.Trainer(
        max_epochs=training_configuration.epochs_count,
        log_every_n_steps=training_configuration.log_interval,
        callbacks=[
            early_stopping,
            model_checkpoint,
        ],
    )

    trainer.fit(
        model=model,
        datamodule=data_module,
    )


## <font style="color:green">5. Model [5 Points]</font>

**Define your model in this section.**

**You are allowed to use any pre-trained model.**

In [5]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import lightning as L

from torchmetrics import MeanMetric
from torchmetrics.classification import MulticlassAccuracy
from torchvision import models
from transformers import ViTForImageClassification


class KenyanFood13Classifier(L.LightningModule):
    """
    KenyanFood13Classifier is the model class for the Kenyan Food 13 Classifier.
    """

    def __init__(
        self,
        num_classes: int = 13,
        weights: str = "DEFAULT",
        unfreeze_layers: int = 4,
        learning_rate: float = 0.001,
    ) -> None:
        """
        Initializes the KenyanFood13Classifier.

        Args:
            num_classes (int, optional): number of classes. Defaults to 13.
            weights (str, optional): weights. Defaults to "DEFAULT".
            learning_rate (float, optional): learning rate. Defaults to 0.001.

        Returns:
            None
        """
        super().__init__()
        self.save_hyperparameters()

        self.model = ViTForImageClassification.from_pretrained(
            "google/vit-base-patch16-224",
            num_labels=num_classes,
            ignore_mismatched_sizes=True,
        )

        for param in self.model.parameters():
            param.requires_grad = False

        for i in range(1, self.hparams.unfreeze_layers):
            for param in self.model.vit.encoder.layer[-i].parameters():
                param.requires_grad = True

        self.mean_train_accuracy = MulticlassAccuracy(
            num_classes=self.hparams.num_classes,
            average="micro",
        )
        self.mean_val_accuracy = MulticlassAccuracy(
            num_classes=self.hparams.num_classes,
            average="micro",
        )

        self.mean_train_loss = MeanMetric()
        self.mean_val_loss = MeanMetric()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): input tensor.

        Returns:
            torch.Tensor: output tensor.
        """
        output = self.model(x)
        return output.logits

    def training_step(
        self,
        batch: torch.Tensor,
        batch_idx: int,
    ) -> torch.Tensor:
        """
        Training step of the model.

        Args:
            batch (torch.Tensor): input tensor.
            batch_idx (int): batch index.

        Returns:
            torch.Tensor: loss tensor.
        """
        data, target = batch
        output = self(data)
        loss = F.cross_entropy(output, target)
        pred_batch = output.detach().argmax(dim=1)

        # Update metrics
        size_of_batch = data.size(0)
        self.mean_train_loss(loss, weight=size_of_batch)
        self.mean_train_accuracy(pred_batch, target)

        self.log(
            "train/batch_loss",
            self.mean_train_loss,
            logger=True,
        )
        self.log(
            "train/batch_acc",
            self.mean_train_accuracy,
            logger=True,
        )

        return loss

    def validation_step(
        self,
        batch: torch.Tensor,
        batch_idx: int,
    ) -> None:
        """
        Validation step of the model.

        Args:
            batch (torch.Tensor): input tensor.
            batch_idx (int): batch index.

        Returns:
            None
        """
        data, target = batch
        output = self(data)
        loss = F.cross_entropy(output, target)
        pred_batch = output.detach().argmax(dim=1)

        # Update metrics
        size_of_batch = data.size(0)
        self.mean_val_loss(loss, weight=size_of_batch)
        self.mean_val_accuracy(pred_batch, target)

    def on_train_epoch_end(self) -> None:
        """
        Called at the end of each training epoch.

        Returns:
            None
        """
        self.log(
            "train/loss",
            self.mean_train_loss,
            logger=True,
        )
        self.log(
            "train/acc",
            self.mean_train_accuracy,
            logger=True,
            prog_bar=True,
        )

    def on_validation_epoch_end(self) -> None:
        """
        Called at the end of each validation epoch.

        Returns:
            None
        """
        self.log(
            "val/loss",
            self.mean_val_loss,
            logger=True,
        )
        self.log(
            "val/acc",
            self.mean_val_accuracy,
            logger=True,
            prog_bar=True,
        )

        self.log(
            "lr",
            self.trainer.optimizers[0].param_groups[0]["lr"],
            logger=True,
            prog_bar=True,
        )

    def configure_optimizers(self) -> optim.Optimizer:
        """
        Configures the optimizer.

        Returns:
            optim.Optimizer: optimizer
        """
        optimizer = optim.Adam(
            self.parameters(),
            lr=self.hparams.learning_rate,
        )

        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode="max",
            patience=3,
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val/acc",
        }


## <font style="color:green">6. Utils [5 Points]</font>

**Define those methods or classes, which have  not been covered in the above sections.**

In [6]:
import pandas as pd

from learn_lib.model import KenyanFood13Classifier
from learn_lib.data_loader import (
    KenyanFood13Dataset,
    KenyanFood13DataModule,
    kaggle_test_images,
)
from learn_lib.config import TrainingConfiguration
from learn_lib.train import train


def main(data_root: str) -> None:
    """Trains the model

    Args:
        data_root (str): path to the data root
    """
    mode = input("Train model? (y/n): ")
    if mode == "y":
        train_model(data_root)
    else:
        checkpoint = input("Enter checkpoint path: ")

        dataset = KenyanFood13Dataset(data_root)
        class_to_idx = dataset.class_to_idx

        model = KenyanFood13Classifier.load_from_checkpoint(checkpoint)
        model.freeze()
        model.eval()

        infer_test_images(
            model,
            data_root,
            class_to_idx,
        )


def train_model(
    data_root: str,
) -> None:
    model = KenyanFood13Classifier()
    train_config = TrainingConfiguration()
    train_config.early_stopping_patience = 10
    datamodule = KenyanFood13DataModule(
        data_root=data_root,
        use_augmentation=True,
    )

    train(
        model=model,
        training_configuration=train_config,
        data_module=datamodule,
    )


def infer_test_images(
    model: KenyanFood13Classifier,
    data_root: str,
    class_to_idx: dict[str, int] = None,
) -> None:
    device = next(model.parameters()).device
    kaggle_test_images_gen = kaggle_test_images(data_root)
    index_to_class = {v: k for k, v in class_to_idx.items()}

    result_dict = {
        "id": [],
        "class": [],
    }

    for image, image_name in kaggle_test_images_gen:
        image = image.to(device)
        output = model(image.unsqueeze(0))
        pred = output.argmax(dim=1)
        pred_class = index_to_class[pred.item()]

        print(f"Image name: {image_name}, Predicted class: {pred_class}")

        result_dict["id"].append(image_name)
        result_dict["class"].append(pred_class)

    result_df = pd.DataFrame(result_dict)

    result_df.to_csv(
        "submission.csv",
        index=False,
    )


## <font style="color:green">7. Experiment [5 Points]</font>

**Choose your optimizer and LR-scheduler and use the above methods and classes to train your model.**

In [8]:
main(data_root="./data")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([13]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([13, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type                      | Params | Mode 
--------------------------------------------------------------------------
0 | model               | ViTForImageClassification | 85.8 M | eval 
1 | mean_train_accuracy | MulticlassAccuracy        | 0      | train
2 | mean_val_accuracy   | 

Epoch 0: 100%|██████████| 164/164 [01:30<00:00,  1.82it/s, v_num=0, val/acc=0.786, lr=0.001]

Metric val/acc improved. New best score: 0.786
Epoch 0, global step 164: 'val/acc' reached 0.78593 (best 0.78593), saving model to '/home/kamal/projects/deep_learning_with_pytorch/week7/checkpoints/model-00.ckpt' as top 1


Epoch 1: 100%|██████████| 164/164 [01:32<00:00,  1.78it/s, v_num=0, val/acc=0.772, lr=0.001, train/acc=0.640]

Epoch 1, global step 328: 'val/acc' was not in top 1


...

Epoch 37: 100%|██████████| 164/164 [01:37<00:00,  1.68it/s, v_num=0, val/acc=0.813, lr=1e-7, train/acc=0.943]

Epoch 37, global step 6232: 'val/acc' was not in top 1


Epoch 38: 100%|██████████| 164/164 [01:37<00:00,  1.69it/s, v_num=0, val/acc=0.813, lr=1e-7, train/acc=0.942]

Monitored metric val/acc did not improve in the last 10 records. Best score: 0.816. Signaling Trainer to stop.
Epoch 38, global step 6396: 'val/acc' was not in top 1


Epoch 38: 100%|██████████| 164/164 [01:37<00:00,  1.69it/s, v_num=0, val/acc=0.813, lr=1e-7, train/acc=0.942]


## <font style="color:green">8. TensorBoard Log [5 Points]</font>

**Share your TensorBoard scalars logs here You can also share (not mandatory) your GitHub link, if you have pushed this project in GitHub.**


<font style="color:red">Note:</font> In light of the recent shutdown of tensorboard.dev, we have updated the submission requirements for your project. Instead of sharing a tensorboard.dev link, you are now required to upload your generated TensorBoard event files directly onto the lab. As an alternative, you may also include a screenshot of your TensorBoard output within your Jupyter notebook. This adjustment ensures that your data visualization and model training efforts are thoroughly documented and accessible for evaluation.

## <font style="color:green">9. Kaggle Profile Link [50 Points]</font>

**Share your Kaggle profile link  with us here to score , points in  the competition.**

**For full points, you need a minimum accuracy of `75%` on the test data. If accuracy is less than `70%`, you gain  no points for this section.**


**Submit `submission.csv` (prediction for images in `test.csv`), in the `Submit Predictions` tab in Kaggle, to get evaluated for  this section.**

> **Kaggle profile link:** [Profile](https://www.kaggle.com/kamalfarahani)